In [5]:
import sys
import subprocess
import numpy
import meshio
import pyvista
pyvista.set_jupyter_backend( "trame" )

path_to_cubit = "/opt/Coreform-Cubit-2023.8/bin"
path_to_exodus = "/home/greg/apps/seacas/lib"
algebra_executable = "/home/greg/apps/seacas/bin/algebra"
goma_executable = "/home/greg/apps/goma/build/goma"

sys.path.append( path_to_cubit )
import cubit
cubit.init( [] )

sys.path.append( path_to_exodus )
import exodus

def main():
    generate_initial_mesh()
    run_goma()
    convert_results()

In [6]:
def run_goma():
    goma_command = f"{goma_executable} -a -i spreading_drop.inp"
    subprocess.check_call( goma_command, shell=True )

def convert_results():
    algebra_command = f"{algebra_executable} out_spreading_drop.e out_spreading_drop_last.e < algebra.in"
    subprocess.run( algebra_command, shell=True )
    meshio_command = "meshio convert out_spreading_drop_last.e out_spreading_drop.xdmf"
    subprocess.check_call( meshio_command, shell=True )

In [7]:
def remesh_droplet():
    ## Remeshing Example
    ## Bob Fermin & Kris Tjiptowidjojo
    ## Avery Dennison
    ## May 2023

    cubit.cmd( "reset" )

    cubit.cmd( "Set Exodus netcdf4 Off " )
    cubit.cmd( "Set Large Exodus Off " )
    cubit.cmd( "Set Overlap Maximum gap 0.0005" )
    cubit.cmd( "Set Node Constraint smart" )
    cubit.cmd( "Set Boolean Regularize on" )
    cubit.cmd( "Set Digits 10" )

    ## The Associativity commands below are
    ## essential for saving on export &
    ## recreating on import the geometry
    ## that needs to be remeshed.
    ##------------------------------
    cubit.cmd( "Set NodeSet Associativity ON" )
    cubit.cmd( "Set Associativity Complete ON" )
    ##------------------------------

    cubit.cmd( "import mesh geometry './Droplet.exoII' feature_angle 135.00  merge  deformed last" )
    cubit.cmd( "delete mesh surface 1 propagate" )

    cubit.cmd( "surface 1 size auto factor 3" )
    cubit.cmd( "mesh surface 1" )

def generate_initial_mesh():
    ## Remeshing Example
    ## Bob Fermin & Kris Tjiptowidjojo
    ## Avery Dennison
    ## May 2023

    cubit.cmd( "reset" )

    cubit.cmd( "Set Exodus netcdf4 Off" )
    cubit.cmd( "Set Large Exodus Off" )
    cubit.cmd( "Set Overlap Maximum gap 0.0005" )
    cubit.cmd( "Set Node Constraint smart" )
    cubit.cmd( "Set Boolean Regularize on" )
    cubit.cmd( "Set Digits 10" )

    ## The Associativity commands below are
    ## essential for saving on export &
    ## recreating on import the geometry
    ## that needs to be remeshed.
    ##------------------------------
    cubit.cmd( "Set NodeSet Associativity ON" )
    cubit.cmd( "Set Associativity Complete ON" )
    ##------------------------------

    Drop_Size = 0.05  # 500 microns

    ## Create a simple droplet resting on a substrate

    cubit.cmd( f"create surface circle radius {Drop_Size} zplane" )
    cubit.cmd( "webcut body 1 with plane yplane offset 0" )
    cubit.cmd( "delete Surface 2" )
    cubit.cmd( "webcut body All with plane xplane offset 0 imprint merge" )
    cubit.cmd( "compress all" )

    ## Mesh the droplet
    cubit.cmd( "surface all size auto factor 5" )
    cubit.cmd( "mesh surface all" )
    cubit.cmd( "surface all smooth scheme centroid area pull free" )
    cubit.cmd( "smooth surface all" )

    ## Add nodesets and sidesets to apply BCs

    cubit.cmd( "nodeset 1 add curve 1" )
    cubit.cmd( "nodeset 1 name 'MeshFence'" )
    cubit.cmd( "Sideset 1 add curve 1" )
    cubit.cmd( "Sideset 1 name 'MeshFence'" )

    cubit.cmd( "nodeset 100 add curve 2 5" )
    cubit.cmd( "nodeset 100 name 'Substrate'" )
    cubit.cmd( "Sideset 100 add curve 2 5" )
    cubit.cmd( "Sideset 100 name 'Substrate'" )

    cubit.cmd( "sideset 101 add curve 2" )
    cubit.cmd( "sideset 101 name 'Substrate-Right'" )
    cubit.cmd( "sideset 102 add curve 5" )
    cubit.cmd( "sideset 102 name 'Substrate-Left'" )

    cubit.cmd( "Nodeset 200 add curve 3 4" )
    cubit.cmd( "Nodeset 200 name 'FreeSurface'" )
    cubit.cmd( "Sideset 200 add curve 3 4" )
    cubit.cmd( "Sideset 200 name 'FreeSurface'" )

    cubit.cmd( "nodeset 1000 add vertex 1" )
    cubit.cmd( "nodeset 1000 name 'CA-Right'" )
    cubit.cmd( "nodeset 2000 add vertex 2" )
    cubit.cmd( "nodeset 2000 name 'CA-Left'" )

    ## Create the material block

    cubit.cmd( "block 1 add surface all" )
    cubit.cmd( "block 1 name 'Liquid'" )

    ## Set the element type

    cubit.cmd( "block 1  element type quad9" )

    ## Export the final mesh
    cubit.cmd( "save cub5 'drop_init_mesh.cub5' overwrite" )
    cubit.cmd( "export mesh 'drop_init_mesh.exoII' dimension 2 overwrite" )
    


In [8]:
main()

...deleting 270 faces from database...
...deleting 570 edges from database...
...deleting 301 nodes from database...
Journaled Command: reset

Journaled Command: set exodus netcdf4 off

Journaled Command: set large exodus file off

Journaled Command: set overlap maximum gap 0.0005

Journaled Command: set node constraint smart

Journaled Command: set boolean regularize on

Coordinates and lengths will be listed with up to 10 digits.
To reset digits to default, use 'set digits -1'
Journaled Command: set digits 10

Nodeset Associativity is On:
Exported mesh files will have data associating each node with geometry.

Journaled Command: set nodeset associativity on

Nodeset Associativity Complete for deformed geometry regeneration is On.

Journaled Command: set associativity complete on

Journaled Command: create surface circle radius 0.05 zplane



Performing webcut operation...
Created volume(s): 2
Updated volume(s): 1
Modified/Created volumes: 1 2 
Journaled Command: webcut body 1 with pl

 (/home/greg/apps/goma/src/mm_input.c:1584)
 (/home/greg/apps/goma/src/bc_special.c:881)



    N e w t o n  C o n v e r g e n c e  - I m p l i c i t   T i m e   S t e p

   ToD   itn   L_oo    L_1     L_2     L_oo    L_1     L_2   lis  asm/slv (sec)  
-------- --- ------- ------- ------- ------- ------- ------- --- --------------- 
22:29:58 [0] 1.0e-03 5.1e-01 1.7e-02 1.4e+03 4.1e+05 2.3e+04  1  1.7e-01/7.0e-02 
22:29:58 [1] 2.7e-04 1.6e-01 4.1e-03 3.3e+02 4.0e+04 1.9e+03  1  1.7e-01/4.0e-02 
22:29:58 [2] 1.1e-03 3.7e-01 1.1e-02 3.9e+02 5.0e+04 2.9e+03  1  1.8e-01/4.0e-02 
22:29:59 [3] 6.1e-04 1.7e-01 5.0e-03 1.8e+02 4.0e+04 1.9e+03  1  1.7e-01/5.0e-02 
22:29:59 [4] 7.7e-04 2.3e-01 6.7e-03 7.5e+01 1.4e+04 6.9e+02  1  1.7e-01/4.0e-02 
22:29:59 [5] 4.5e-04 9.7e-02 3.2e-03 3.2e+01 5.3e+03 2.1e+02  1  1.8e-01/4.0e-02 
22:29:59 [6] 2.1e-04 3.4e-02 1.1e-03 7.8e+00 1.2e+03 3.9e+01  1  1.8e-01/5.0e-02 
22:29:59 [7] 2.5e-05 5.3e-03 1.7e-04                         skp 1.9e-01/0.0e+00 
-------- --- ------- ------- ------- ------- ------- ------- --- --------------- 
scaled solution no

/bin/sh: 1: cannot open algebra.in: No such file
Traceback (most recent call last):
  File "/home/greg/.local/bin/meshio", line 8, in <module>
    sys.exit(main())
  File "/home/greg/.local/lib/python3.10/site-packages/meshio/_cli/_main.py", line 52, in main
    return args.func(args)
  File "/home/greg/.local/lib/python3.10/site-packages/meshio/_cli/_convert.py", line 53, in convert
    mesh = read(args.infile, file_format=args.input_format)
  File "/home/greg/.local/lib/python3.10/site-packages/meshio/_helpers.py", line 71, in read
    return _read_file(Path(filename), file_format)
  File "/home/greg/.local/lib/python3.10/site-packages/meshio/_helpers.py", line 90, in _read_file
    raise ReadError(f"File {path} not found.")
meshio._exceptions.ReadError: File out_spreading_drop_last.e not found.


CalledProcessError: Command 'meshio convert out_spreading_drop_last.e out_spreading_drop.xdmf' returned non-zero exit status 1.

In [ ]:


results = pyvista.read( "./out_spreading_drop.xdmf" )
displacement = numpy.transpose( numpy.array( [ results["DMX"], results["DMY"], numpy.zeros( len(results["DMX"]) ) ] ) )
results.add_field_data( displacement, "disp" )
warped = results.warp_by_vector( vectors="disp", factor=1.0 )
warped.plot( scalars=results['disp'], cpos='xy' )